In [95]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_recall_fscore_support

In [96]:
# 1. 데이터 불러오기
data = pd.read_csv('azdigar nav sartirovka+0 delete.csv')

In [97]:
# 분석할 열들
columns = [
    'w08chronic_a', 'w08chronic_b', 'w08chronic_c',
    'w08chronic_d', 'w08chronic_e', 'w08chronic_f',
    'w08chronic_g', 'w08chronic_h', 'w08chronic_i',
    'w08chronic_k', 'w08chronic_l', 'w08chronic_m'
] 

In [98]:
# 2. 데이터 클리닝 - 'w08chronic_m'에서 값이 3인 행 제거
cleaned_data = data[data['w08chronic_m'] != 3] 


In [99]:
# # 3. 피처와 타겟 정의
# X = cleaned_data.drop(columns=columns)
# y = cleaned_data[columns].values
# 3. 피처 데이터와 타겟 데이터 정의
X = cleaned_data.drop(columns=columns)
y_all = cleaned_data[columns]

In [100]:

# 4. 데이터 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [94]:
# 멀티라벨을 멀티-핫 인코딩으로 변환
mlb = MultiLabelBinarizer()
y_binarized = mlb.fit_transform(y)

In [105]:
# 5. 유전 알고리즘 파라미터 설정
population_size = 10  # 개체군 크기
num_generations = 10  # 세대 수
mutation_rate = 0.05  # 돌연변이 확률

# 초기 개체군 생성
def initialize_population():
    return [np.random.randint(2, size=X_scaled.shape[1]).tolist() for _ in range(population_size)]

# 적합도 평가 함수
def fitness_function(individual, model, X_train, y_train, X_test, y_test):
    selected_features = [i for i, bit in enumerate(individual) if bit]
    if len(selected_features) == 0:
        return 0  # 피처가 선택되지 않으면 적합도는 0
    
    X_train_selected = X_train[:, selected_features]
    X_test_selected = X_test[:, selected_features]
    
    try:
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)

        # 타겟 클래스가 두 개 이상이면 AUC 계산을 위해 타겟을 이진화 (One-vs-Rest 방식)
        y_test_bin = label_binarize(y_test, classes=np.unique(y_train))
        if y_test_bin.shape[1] == 1:  # 클래스가 하나인 경우
            y_test_bin = np.concatenate([1 - y_test_bin, y_test_bin], axis=1)
        
        f1 = f1_score(y_test, y_pred, average='macro')
        return f1  # 적합도는 F1 점수로 설정
    except Exception as e:
        print(f"적합도 평가 중 오류 발생: {e}")
        return 0

# 개체군 선택 함수
def selection(population, model, X_train, y_train, X_test, y_test):
    scores = [fitness_function(ind, model, X_train, y_train, X_test, y_test) for ind in population]
    selected_indices = np.argsort(scores)[-population_size // 2:]
    return [population[i] for i in selected_indices]

# 교차 함수
def crossover(parent1, parent2):
    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

# 돌연변이 함수
def mutate(individual):
    for i in range(len(individual)):
        if random.random() < mutation_rate:
            individual[i] = 1 - individual[i]  # 비트 반전
    return individual

# 유전 알고리즘 실행 함수
def run_genetic_algorithm(model, X_train, y_train, X_test, y_test):
    population = initialize_population()
    best_individual = None
    best_f1 = -1

    for generation in range(num_generations):
        print(f"Generation {generation + 1}/{num_generations}")
        selected_population = selection(population, model, X_train, y_train, X_test, y_test)
        
        children = []
        while len(children) < population_size - len(selected_population):
            parent1, parent2 = random.sample(selected_population, 2)
            child1, child2 = crossover(parent1, parent2)
            child1 = mutate(child1)
            child2 = mutate(child2)
            children.extend([child1, child2])
        
        population = selected_population + children[:population_size - len(selected_population)]
        
        for ind in population:
            current_f1 = fitness_function(ind, model, X_train, y_train, X_test, y_test)
            if current_f1 > best_f1:
                best_f1 = current_f1
                best_individual = ind

    return best_individual

In [106]:
# 6. 결과 저장을 위한 빈 데이터프레임 생성
result_table = pd.DataFrame(columns=[
    'Target', 'Model', 'Precision', 'Recall', 'F1-Score', 
    'Accuracy', 'Micro F1', 'Macro F1', 'Weighted F1', 'AUC'
])

# 7. 각 타겟 열을 사용한 학습 및 평가
for target_column in columns:
    print(f'\n[{target_column}] 열을 예측:')
    print('==================================')

    y = cleaned_data[target_column].values
    y_binarized = label_binarize(y, classes=np.unique(y))  # 타겟 이진화

    # 데이터 나누기
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_binarized, test_size=0.2, random_state=42)

    # 모델 정의
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    multi_target_rf = MultiOutputClassifier(rf, n_jobs=-1)

    # 유전 알고리즘을 사용한 피처 선택
    best_individual = run_genetic_algorithm(multi_target_rf, X_train, y_train, X_test, y_test)
    selected_features = [i for i, bit in enumerate(best_individual) if bit]

    if len(selected_features) == 0:
        print(f'선택된 피처가 없습니다. {target_column} 열에 대한 모델 평가를 건너뜁니다.')
        continue

    # 선택된 피처만 사용
    X_train_selected = X_train[:, selected_features]
    X_test_selected = X_test[:, selected_features]

    # 모델 학습
    multi_target_rf.fit(X_train_selected, y_train)

    # 예측
    y_pred = multi_target_rf.predict(X_test_selected)

    # 평가
    accuracy = accuracy_score(y_test, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
    micro_f1 = f1_score(y_test, y_pred, average='micro')
    macro_f1 = f1_score(y_test, y_pred, average='macro')
    weighted_f1 = f1_score(y_test, y_pred, average='weighted')

    # AUC 계산
    try:
        auc = roc_auc_score(y_test, multi_target_rf.predict_proba(X_test_selected), multi_class="ovr", average='weighted')
    except ValueError:
        auc = np.nan

    # 결과 저장
    new_row = pd.DataFrame({
        'Model': ['Random Forest'],
        'Target': [target_column],
        'Precision': [precision],
        'Recall': [recall],
        'F1-Score': [f1],
        'Accuracy': [accuracy],
        'Micro F1': [micro_f1],
        'Macro F1': [macro_f1],
        'Weighted F1': [weighted_f1],
        'AUC': [auc]
    })
    result_table = pd.concat([result_table, new_row], ignore_index=True)



[w08chronic_a] 열을 예측:
Generation 1/10
Generation 2/10
Generation 3/10
Generation 4/10
Generation 5/10
Generation 6/10
Generation 7/10
Generation 8/10
Generation 9/10
Generation 10/10

[w08chronic_b] 열을 예측:
Generation 1/10
Generation 2/10
Generation 3/10
Generation 4/10
Generation 5/10
Generation 6/10
Generation 7/10
Generation 8/10
Generation 9/10
Generation 10/10

[w08chronic_c] 열을 예측:
Generation 1/10
Generation 2/10
Generation 3/10
Generation 4/10
Generation 5/10
Generation 6/10
Generation 7/10
Generation 8/10
Generation 9/10
Generation 10/10


c:\Users\azamj\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



[w08chronic_d] 열을 예측:
Generation 1/10
Generation 2/10
Generation 3/10
Generation 4/10
Generation 5/10
Generation 6/10
Generation 7/10
Generation 8/10
Generation 9/10
Generation 10/10


c:\Users\azamj\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



[w08chronic_e] 열을 예측:
Generation 1/10
Generation 2/10
Generation 3/10
Generation 4/10
Generation 5/10
Generation 6/10
Generation 7/10
Generation 8/10
Generation 9/10
Generation 10/10


...

c:\Users\azamj\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



[w08chronic_l] 열을 예측:
Generation 1/10
Generation 2/10
Generation 3/10
Generation 4/10
Generation 5/10
Generation 6/10
Generation 7/10
Generation 8/10
Generation 9/10
Generation 10/10


c:\Users\azamj\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



[w08chronic_m] 열을 예측:
Generation 1/10
Generation 2/10
Generation 3/10
Generation 4/10
Generation 5/10
Generation 6/10
Generation 7/10
Generation 8/10
Generation 9/10
Generation 10/10


c:\Users\azamj\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [107]:
# 8. 결과 테이블 출력
print("\n전체 모델 평가 결과:")
print(result_table)


전체 모델 평가 결과:
          Target          Model  Precision    Recall  F1-Score  Accuracy  \
0   w08chronic_a  Random Forest   0.770030  0.750552  0.747971  0.750552   
1   w08chronic_b  Random Forest   0.718131  0.770419  0.688530  0.770419   
2   w08chronic_c  Random Forest   0.859611  0.927152  0.892105  0.927152   
3   w08chronic_d  Random Forest   0.941285  0.970199  0.955523  0.970199   
4   w08chronic_e  Random Forest   0.943428  0.971302  0.957163  0.971302   
5   w08chronic_f  Random Forest   0.799307  0.894040  0.844024  0.894040   
6   w08chronic_g  Random Forest   0.950821  0.948124  0.924912  0.948124   
7   w08chronic_h  Random Forest   0.953796  0.951435  0.928813  0.951435   
8   w08chronic_i  Random Forest   0.766877  0.779249  0.757820  0.779249   
9   w08chronic_k  Random Forest   0.971508  0.985651  0.978529  0.985651   
10  w08chronic_l  Random Forest   0.973685  0.986755  0.980177  0.986755   
11  w08chronic_m  Random Forest   0.982418  0.991170  0.986775  0.991170  